In [6]:
from ollama import Client
from langchain_core.prompts import PromptTemplate
import json
from langchain_ollama.llms import OllamaLLM
from langchain.chains.llm import LLMChain
from langchain_core.runnables import RunnablePassthrough

In [11]:
# Subtopics chain

subtopics_template = PromptTemplate.from_template(
    """Generate a list of {number_of_subtopics} subheadings for a highschool essay on {topic}.
    Order the subheadings such that they have a logical sequence.
    Each subheading should be accompanied with a detailed discription on what the contents should be about.
    Provide output in json format as follows:

    {{'<heading1>':'<description1>',
    '<heading2>':'<description2>',...}}
    
    Only return the json object without any additional text."""
)
    
model = OllamaLLM(model="llama3.2:latest", run_local=True, format='json')

subtopic_chain = LLMChain(
    llm=model,
    prompt=subtopics_template,
    output_key="subtopics",
)

In [17]:
# subtopic_chain.invoke(
#     {
#         "topic": "History of the Internet",
#         "number_of_subtopics": 3
#     }
# )

In [21]:
passthrough_chain = RunnablePassthrough.assign(
    subtopic_headings = lambda x: list(json.loads(x['subtopics']).keys()),
    subtopic_descriptions = lambda x: list(json.loads(x['subtopics']).values())
)

In [23]:
# passthrough_chain.invoke(subtopic_chain.invoke(
#     {
#         "topic": "History of the Internet",
#         "number_of_subtopics": 3
#     }
# ))

In [24]:
sequence_chain = subtopic_chain | passthrough_chain
result = sequence_chain.invoke(
    {
        "topic": "History of the Internet",
        "number_of_subtopics": 3
    }
)
result

{'topic': 'History of the Internet',
 'number_of_subtopics': 3,
 'subtopics': '{"Prehistory of the Internet": "This section will explore the early beginnings of the internet, including the development of the first network, ARPANET, and how it laid the foundation for the modern internet. It will also discuss other key players and events that contributed to the growth of the internet during this period."}',
 'subtopic_headings': ['Prehistory of the Internet'],
 'subtopic_descriptions': ['This section will explore the early beginnings of the internet, including the development of the first network, ARPANET, and how it laid the foundation for the modern internet. It will also discuss other key players and events that contributed to the growth of the internet during this period.']}